In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('hexoskin_2bands_training_set_10_final.xlsx')
df

,exp_diff_mean,A_exp_dur_sd,A_exp_diff_mean,exp_dur_sd,duty_cycle_mean,A_RRV_RMSSD,RRV_RMSSD,ie_ratio_mean,A_duty_cycle_mean,insp_flow_sd,Task_Label,Participant,Classification
0,0.127790,0.164364,0.117188,0.156604,0.492097,404.771735,376.386712,0.971761,0.492399,1.445912e+06,12.0a,10785,0
1,0.201451,0.174588,0.241071,0.152211,0.486992,392.384431,351.137516,0.950939,0.485336,1.034691e+06,12.0b,10785,0
2,0.199219,0.251690,0.236719,0.213642,0.476697,459.428807,457.608476,0.913378,0.482721,1.890795e+06,12.0c,10785,0
3,0.285156,0.197705,0.266276,0.213173,0.477750,450.569753,439.827676,0.918119,0.484587,1.316481e+06,12.0d,10785,0
4,0.159375,0.193377,0.108073,0.213828,0.481671,516.192869,457.645154,0.936229,0.488034,9.634884e+05,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,0.203125,0.209238,0.218316,0.200555,0.490259,357.971103,334.175635,0.967894,0.488741,2.209139e+06,85,98586,0
2900,0.921387,0.731201,0.941894,0.725393,0.401725,1042.040237,1209.145284,0.933997,0.431877,1.280213e+07,87,98586,1
2901,0.106337,0.115003,0.101562,0.117894,0.496388,153.331025,185.358320,0.987026,0.494804,2.990174e+06,89,98586,0
2902,2.145182,1.115591,1.260045,1.447558,0.412197,2387.940073,3053.223554,0.748645,0.360902,1.441962e+07,91,98586,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[1.27790200e-01, 1.64364500e-01, 1.17187500e-01, ...,
         9.71761000e-01, 4.92398700e-01, 1.44591181e+06],
        [2.01450900e-01, 1.74588400e-01, 2.41071400e-01, ...,
         9.50939100e-01, 4.85336100e-01, 1.03469140e+06],
        [1.99218800e-01, 2.51689700e-01, 2.36718800e-01, ...,
         9.13377900e-01, 4.82720700e-01, 1.89079541e+06],
        ...,
        [1.06336800e-01, 1.15003100e-01, 1.01562500e-01, ...,
         9.87025600e-01, 4.94804200e-01, 2.99017395e+06],
        [2.14518230e+00, 1.11559080e+00, 1.26004460e+00, ...,
         7.48644500e-01, 3.60902200e-01, 1.44196160e+07],
        [1.23535200e-01, 1.63306400e-01, 1.32812500e-01, ...,
         9.26293000e-01, 4.83311700e-01, 2.04013979e+06]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['exp_diff_mean',
  'A_exp_dur_sd',
  'A_exp_diff_mean',
  'exp_dur_sd',
  'duty_cycle_mean',
  'A_RRV_RMSSD',
  'RRV_RMSSD',
  'ie_ratio_mean',
  'A_duty_cycle_mean',
  'insp_flow_sd'],
 'partici

In [4]:
#running the nested cross-validation for the Accelerometer method using Gradient Boosting

from sklearn.metrics import accuracy_score, precision_score, recall_score

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'subsample': [0.9, 1.0],  
    'max_depth': [3, 4],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2', None]
}


gbc = GradientBoostingClassifier(random_state=random_seed)

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=gbc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

nested_accuracy_scores:    [0.95454545 0.87878788 0.92424242 0.87878788 0.98484848 0.95454545
 1.         0.89393939 0.93939394 0.96969697 0.98484848 0.68181818
 0.77272727 0.90909091 0.84848485 0.90909091 1.         0.83333333
 0.87878788 0.90909091 0.89393939 0.95454545 0.83333333 0.89393939
 0.84848485 0.95454545 0.95454545 1.         0.83333333 0.75757576
 0.89393939 0.93939394 0.86363636 0.96969697 0.84848485 0.92424242
 0.90909091 0.89393939 0.98484848 0.92424242 0.81818182 0.90909091
 0.86363636 0.95454545]
mean score:            0.9029
nested_precision_scores:    [0.95757576 0.89818182 0.93230174 0.88234266 0.98520085 0.95757576
 1.         0.89337255 0.94008264 0.97202797 0.98545455 0.7085044
 0.82387935 0.91264822 0.85353535 0.91223776 1.         0.84033613
 0.88063241 0.91223776 0.91788856 0.95757576 0.87052342 0.89466089
 0.86767677 0.95757576 0.95459141 1.         0.87052342 0.75413223
 0.90756587 0.94805195 0.88797654 0.97202797 0.85244755 0.93730408
 0.92045455 0.8933725

In [5]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.8750
Specificity: 1.0000
Fold 2:
Sensitivity: 0.6667
Specificity: 1.0000
Fold 3:
Sensitivity: 0.7917
Specificity: 1.0000
Fold 4:
Sensitivity: 0.8750
Specificity: 0.8810
Fold 5:
Sensitivity: 0.9583
Specificity: 1.0000
Fold 6:
Sensitivity: 0.8750
Specificity: 1.0000
Fold 7:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 8:
Sensitivity: 0.8333
Specificity: 0.9286
Fold 9:
Sensitivity: 0.8750
Specificity: 0.9762
Fold 10:
Sensitivity: 1.0000
Specificity: 0.9524
Fold 11:
Sensitivity: 1.0000
Specificity: 0.9762
Fold 12:
Sensitivity: 0.7083
Specificity: 0.6667
Fold 13:
Sensitivity: 0.9167
Specificity: 0.6905
Fold 14:
Sensitivity: 0.7917
Specificity: 0.9762
Fold 15:
Sensitivity: 0.6667
Specificity: 0.9524
Fold 16:
Sensitivity: 0.9167
Specificity: 0.9048
Fold 17:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 18:
Sensitivity: 0.6250
Specificity: 0.9524
Fold 19:
Sensitivity: 0.7500
Specificity: 0.9524
Fold 20:
Sensitivity: 0.9167
Specificity: 0.9048
Fold 21:
Sensitivity: 1.0000


In [6]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")

test_accuracy: Mean=0.902892561983471, SD=0.06825563163036329, 95% CI=(0.8827246822572856, 0.9230604417096564), Min=0.6818181818181818, Max=1.0
test_precision: Mean=0.9110067239185462, SD=0.06228527205411454, 95% CI=(0.8926029402966638, 0.9294105075404286), Min=0.7085043988269796, Max=1.0
test_recall: Mean=0.902892561983471, SD=0.06825563163036329, 95% CI=(0.8827246822572856, 0.9230604417096564), Min=0.6818181818181818, Max=1.0
test_f1_score: Mean=0.8952328384620775, SD=0.0724292650220067, 95% CI=(0.8738317515374049, 0.91663392538675), Min=0.6727272727272726, Max=1.0
test_roc_auc: Mean=0.963068181818182, SD=0.04746299547989443, 95% CI=(0.9490440205670366, 0.9770923430693274), Min=0.7609126984126984, Max=1.0
sensitivity_scores: Mean=0.8759469696969698, SD=0.11489822354392365, 95% CI=(0.8419973380279635, 0.9098966013659762), Min=0.625, Max=1.0
specificity_scores: Mean=0.9182900432900433, SD=0.08629031974426479, 95% CI=(0.8927933516828082, 0.9437867348972784), Min=0.6666666666666666, Max=

In [7]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')


Fold 1 - Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 1.0}
Fold 2 - Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 100, 'subsample': 0.9}
Fold 3 - Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.9}
Fold 4 - Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 200, 'subsample': 1.0}
Fold 5 - Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 200, 'subsample': 1.0}
Fold 6 - Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'sqrt', 'min_s

In [8]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')


Hyperparameter: learning_rate - Most frequent value: 0.1 (Count: 44)
Hyperparameter: max_depth - Most frequent value: 3 (Count: 28)
Hyperparameter: max_features - Most frequent value: None (Count: 24)
Hyperparameter: min_samples_leaf - Most frequent value: 1 (Count: 32)
Hyperparameter: min_samples_split - Most frequent value: 2 (Count: 32)
Hyperparameter: n_estimators - Most frequent value: 100 (Count: 27)
Hyperparameter: subsample - Most frequent value: 0.9 (Count: 27)
